## Set up and read the input file

We first import all necessary classes and read the input file. While reading the input file, we count the observation using the matrix model by supplying the class CountMatrixModel.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import inputFile.inputFile as inputFile
import spoke_model.countMatrixModel as cmm
import meanfield.simulateLikelihood as smlt

fileName = "gavin2002.csv"
print('Hello, ' + fileName)
inputSet = inputFile.CInputSet(fileName, cmm.CountMatrixModel)

## Parameter search

To search for a good $\psi$, we run a simple hill-climbing search until no further improvement is found.
We compute both the negative log-likelihood and the AIC. 

In [ ]:
from time import time as timer

def hill_climbing(inputSet, Nk, step=0.5):

    nProteins = inputSet.observationG.nProteins
    cmfa = smlt.CMeanFieldAnnealing(nProteins, Nk) # default

    funcInfer = cmfa        

    funcInfer.estimate(inputSet.observationG, nProteins, Nk, 0.3)
    (fn, fp, errs, f_last) = funcInfer.computeErrorRate(inputSet.observationG, nProteins)
    x_values = np.arange(1.0, 10.5, step)
    y_values = np.zeros(len(x_values), dtype=float)
    aics = np.zeros(len(x_values), dtype=float) 
    for i, psi in enumerate(x_values):
        ts = timer()
        f_value = funcInfer.estimate(inputSet.observationG, nProteins, Nk, psi) 
        te = timer()
        print("Time running MFA: ", te-ts)
        print("x = ", psi, "f(x) = ", f_value)
        (fn, fp, errs, likelihood) = funcInfer.computeErrorRate(inputSet.observationG, nProteins)
        print("\tLikelihood =", likelihood)
        y_values[i] = likelihood
        aics[i] = (Nk - likelihood)/(Nk - f_last)
        f_last = likelihood

    return (x_values, aics, y_values)

x_values, aics, y_values = hill_climbing(inputSet, 300, step=0.2)

## Plot the negative log-likelihood and the AIC

We first filter the raw data, before we locate the inflection points. Here, the inflection points define values where the gradient change direction.
Since $\psi$ and the number of clusters are related, we will later fix $\psi$ and run another search on a number of clusters.
Smaller $\psi$ results in splitting up clusters, while larger $\psi$ tends to merge clusters.

In [ ]:
%matplotlib inline

In [ ]:
from scipy.ndimage import gaussian_filter1d
aics_filter = gaussian_filter1d(aics, 1)
y_values = y_values/np.max(y_values)
y_filter = gaussian_filter1d(y_values, 1)
d2 = np.gradient(np.gradient(y_filter))
aics_d2 = np.gradient(np.gradient(aics_filter))
infls = np.where(np.diff(np.sign(aics_d2)))[0]
print("psi = ", x_values[infls])

plt.plot(x_values, aics_filter, label='AIC Filter')
plt.plot(x_values, aics, label='AIC')
for i, infl in enumerate(infls):
    plt.axvline(x=x_values[infl], color='k')
plt.legend(bbox_to_anchor=(1.5, 1.0))
plt.show()

## Number of clusters

In this example, we set $\psi$ to be the first inflection points and search for a number of clusters by plotting the log-likelihood (scaled).

In [ ]:
def clustering(inputSet, k, psi):
    nProteins = inputSet.observationG.nProteins
    cmfa = smlt.CMeanFieldAnnealing(nProteins, k) # default

    funcInfer = cmfa        

    funcInfer.estimate(inputSet.observationG, nProteins, k, psi)
    (fn, fp, errs, f_last) = funcInfer.computeErrorRate(inputSet.observationG, nProteins)
    return f_last

ks = np.arange(100, 700, 100)
for infl in infls[:3]:
    ls = []
    for i, k in enumerate(ks):
        ls.append(clustering(inputSet, k, x_values[infl]))
    ls = ls/np.max(ls)
    plt.plot(ks, ls)

plt.show()